This script is a first experiment in analyzing multi-hazard impacts on the power grid

It uses the following approach:
- Identify power grid disturbances across power grid data sets
- Identify periods of 'extreme' disruption
- Selects data in a given area of interest from three hazards
    - wildfire
    - space weather
    - terrestrial weather (wind, precipitation, rain)
- Visualizes everything on a map
- Identifies simple compounding connections (same space and time)


Start with time periods
- 2014-2016 (balance of availability in power grid outage data, space weather activity, GIC data)


Start with area of interest
- California


In [ ]:
import numpy as np
import pandas as pd
import os, sys

import geopandas as gpd

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter


### Power grid disturbance data

Candidates
- [new dataset from EAGLE-I](https://www.nature.com/articles/s41597-024-03095-5) (Data are available in the Figshare repository at https://doi.org/10.6084/m9.figshare.24237376) 
- [DOE electric disturbance events OE-417](https://www.oe.netl.doe.gov/OE417_annual_summary.aspx)


In [ ]:
# Read in 2014 data
outage_directory = '/Users/ryanmc/Documents/Conferences/Jack_Eddy_Symposium_2022/dev/outage_data/'
oe417 = pd.read_excel(os.path.join(outage_directory,'DOE-OE-417/2014_Annual_Summary.xlsx'),header=1)
oe417 = oe417.dropna(subset=['Time Event Began'])
oe417 = oe417.iloc[:-2]
# eaglei = pd.read_csv(os.path.join(outage_directory,'EAGLE-I/eaglei_outages_2014.csv'))


In [ ]:
oe417

In [ ]:
oe417 = oe417[oe417['Area Affected'].str.contains('California')]

In [ ]:
eaglei = eaglei[eaglei['state'] == 'California']

In [ ]:
for o in range(len(oe417['Number of Customers Affected'])):
    if oe417['Number of Customers Affected'].iloc[o]=='Unknown':
        oe417['Number of Customers Affected'].iloc[o] = np.nan
    
        

In [ ]:
# Apply severity filters
oe417 = oe417[oe417['Number of Customers Affected']>1000]
eaglei = eaglei[eaglei['customers_out']>1000]


In [ ]:
# Identify spatial areas of these candidate events (define the bounds of them as the regions they 
#   covered and consruct geopandas dataframes for them)

# oe417
gdf_states = gpd.read_file('/Users/ryanmc/Documents/Conferences/Jack_Eddy_Symposium_2022/dev/location_data/Census_Bureau_Data/tl_2014_us_state/tl_2014_us_state.shp')
gdf_counties = gpd.read_file('/Users/ryanmc/Documents/Conferences/Jack_Eddy_Symposium_2022/dev/location_data/Census_Bureau_Data/tl_2014_us_county/tl_2014_us_county.shp')



In [ ]:
state = oe417['Area Affected'].iloc[-1]
state

In [ ]:
# Filter the GeoDataFrame by state name (example for 'California')
state_polygon = gdf_states[gdf_states['NAME'] == 'California']

# Display the polygon geometry
print(state_polygon.geometry)

state_polygon.plot()

In [ ]:
gdf_counties

In [ ]:
# Filter by both state and county name (example for 'Los Angeles County' in 'California')
county_polygon = gdf_counties[(gdf_counties['STATEFP'] == '06') & (gdf_counties['COUNTYFP']=='075')]

# Display the polygon geometry
print(county_polygon.geometry)

county_polygon.plot()

### Power grid physical network
Get the data and limit to the area affected by the outage (right now only manually gets the California power grid data)

In [ ]:
file_path = "/Users/ryanmc/Documents/Conferences/Jack_Eddy_Symposium_2022/dev/physical_grid_data/U.S._Electric_Power_Transmission_Lines.geojson"
gdf = gpd.read_file(file_path)
# gdf = gdf.to_crs(epsg=3857)
# gdf = gdf.to_crs(epsg=4326)

 

# Convert timeframes to folium-friendly types
gdf['SOURCEDATE'] = pd.to_datetime(gdf['SOURCEDATE']).dt.strftime('%Y-%m-%dT%H:%M:%S')
gdf['VAL_DATE'] = pd.to_datetime(gdf['VAL_DATE']).dt.strftime('%Y-%m-%dT%H:%M:%S')


In [ ]:
# Load California shape file
ca_boundaries_file = '/Users/ryanmc/Documents/Conferences/Jack_Eddy_Symposium_2022/dev/location_data/ca_state/CA_State.shp'
ca_boundaries = gpd.read_file(ca_boundaries_file)
    
    
ca_boundaries = ca_boundaries.to_crs('EPSG:4326')


In [ ]:
# Assuming your GeoPandas DataFrame is named 'gdf', filter the data within California
data_within_california = gdf[gdf.geometry.within(ca_boundaries.geometry.iloc[0])]

# # Plot the data within California
# ax = ca_boundaries.plot(color='lightgray', edgecolor='black')
# data_within_california.plot(ax=ax, color='blue', markersize=5)

map = data_within_california.plot(figsize=(20, 16),linewidth=0.5)

# Define common voltages and corresponding colors
common_voltages = [0, 44, 115, 161, 169, 230, 345]

hex_colors = ['blue','green', 'cyan', 'yellow', 'orange', 'red']
legend_labels = ['<' + str(i) for i in common_voltages[1:]]

# Plot overlays for each common voltage
for voltage, color, label in zip(common_voltages, hex_colors, legend_labels):
    overlay = data_within_california[data_within_california['VOLTAGE'] > voltage]  # Filter GeoDataFrame based on voltage
    overlay.plot(ax=map, marker='o', color=color, markersize=5, linewidth=0.5, label=label)
    
# Create a custom legend with colored patches
legend_handles = [Patch(color=color, label=label) for color, label in zip(hex_colors, legend_labels)]

# Add legend
map.legend(handles=legend_handles, title='Voltage', loc='upper right')

# Set xy axis limits
map.set_xlim((-127, -105))
map.set_ylim((30,45))

# Put labels on the map
map.set_xlabel("Longitude")
map.set_ylabel("Latitude")
map.set_title("Major California Transmission Lines")


### Space Weather data

Candidates
- substorms events lists
- supermag optimally interpolated maps
- areas of aurora
- magnetometers in AoI above thresholds
- GICs directly



For now exploring, manually, a potential connection between concurrent space weather and Camp Fire events (November 8, 2018)


In [ ]:
spwx_dir = '/Users/ryanmc/Documents/Conferences/Jack_Eddy_Symposium_2022/dev/risk-resiliency-spwx/data/event_20180825/'
GIC_ids = ['10116','10113','10119']
location_data = pd.read_csv(os.path.join(spwx_dir,'GIC/gic_monitors.csv'))
filtered_data = location_data[location_data['Device ID'].astype(str).isin(GIC_ids)]

# Display the filtered data to ensure the correct rows are selected
print(filtered_data[['Device ID', ' Latitude', ' Longitude']])




In [ ]:
# Load a shapefile of the continental US (or download one)
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Filter to the continental US (excludes Alaska and Hawaii)
us_map = world[(world.name == "United States of America")]

# Plot the continental US
fig, ax = plt.subplots(figsize=(10, 8))
us_map.plot(ax=ax, color='lightgray')

# Plot the filtered GIC IDs
ax.scatter(filtered_data[' Longitude']*-1, filtered_data[' Latitude'], color='red', s=100, label='GIC IDs')

# Add labels and title
plt.title('GIC Monitor Locations on Continental US Map')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.legend()

# Show the plot
plt.show()


In [ ]:
gic_data = pd.DataFrame()
for g in GIC_ids:
    file_tmp = os.path.join(spwx_dir,'GIC/2018E01_'+g+'.csv')
    print(file_tmp)
    data_tmp = pd.read_csv(file_tmp)
    gic_data[g] = data_tmp['GICMeasured']
    gic_data[g+' datetime'] = data_tmp['SampleDateTime']
    gic_data[g+' datetime'] = pd.to_datetime(gic_data[g+' datetime'], format='%m/%d/%Y %I:%M:%S %p')
    gic_data
    
    # Plot the time series
    plt.plot(gic_data[g+' datetime'], gic_data[g], label=f'GIC {g}', linewidth=2)

# Format the x-axis to handle datetime intelligently
ax = plt.gca()  # Get current axis
ax.xaxis.set_major_locator(mdates.AutoDateLocator())  # Automatically adjust date ticks
ax.xaxis.set_major_formatter(mdates.AutoDateFormatter(mdates.AutoDateLocator()))  # Format date labels
# Set custom date format (e.g., 'Month-Day Year')
ax.xaxis.set_major_formatter(DateFormatter('%b %d, %Y'))


# Rotate date labels to avoid overlap
plt.gcf().autofmt_xdate()

# Add labels and title
plt.xlabel('Time')
plt.ylabel('Measurement')
plt.title('Time Series of GIC Monitors')
plt.legend()



# Display the plot
plt.show()

In [ ]:
gic_data['10116 datetime'][0]

### Terrestrial Weather data

### Wildfire data

### Visualize spatial areas on common map